In [ ]:
from pyraysum import prs, Geometry, Model, Control
import numpy as np
import os
import obspy as op
import pandas as pd

In [ ]:
def all(baz, slow, thickn, rho, vp, vs, dip, strike, plunge, trend, ani, npts, dt, scale = 20, print_model = False, plot_model = False, plot_geom = False):
    thickn = np.array(thickn) * 1000
    flag = [1 if i == 0 else 0 for i in ani]
    model = Model(thickn, rho, vp, vs=vs, strike=strike, dip=dip, flag=flag, plunge=plunge, trend=trend, ani = ani)
    if print_model:
        print("The Model configuration is as below:")
        print(model)
    # print(plunge, Trend, ani)
    geom = Geometry(baz, slow)
    
    rc = Control(rot=1, mults=1, verbose=False, npts=npts, dt=dt)
    result = prs.run(model, geom, rc, rf=True)
    result.filter('rfs', 'lowpass', freq=1., zerophase=True, corners=2)
    if plot_model:
        model.plot()
        result.plot('rfs', tmin=-0.5, tmax = npts*dt*0.5, scale = scale)
    if plot_geom:
        geom.plot()
    
    return model, geom, result

In [ ]:
baz = np.arange(0., 360., 6.)
slow = 0.003
thickn = [40, 3, 0]   #km
rho = [2800, 2800, 2800]
vp = [6400, 7000, 7800]
vs = [3600, 3800, 4000]
#plane geometry
dip = [0, 5, -0]       #degrees
strike = [0, -0, -0]
#ansitropy
plunge = [0, 0, 0]
trend = [0, 0, 0]
ani = [0, 0, 0]
npts = 1500
dt = 0.0125

model, geom, result = all(baz, slow, thickn, rho, vp, vs, dip, strike, plunge, trend, ani, npts, dt, scale = 500, print_model = False, plot_model = False)

In [ ]:
from codes_RF.prs_utils import *

X, time = harm_dec_calc(result, sig_len = 1500)
plot_har_dec(X, time)

In [ ]:
def read_all_data(keyword):
    baz = []
    slow = []
    path = "DATA/waveforms_list.csv"
    waveforms_list = pd.read_csv(path)
    filtered_files = waveforms_list[waveforms_list['sta_code']==keyword].copy()
    filtered_files = filtered_files[filtered_files['rf_quality'] == 1].copy()
    path = "DATA/RF/"
    npts = 426 * 2
    dt = 0.2
    data_real = np.zeros((len(filtered_files), npts))
    idx = 0
    for index, row in filtered_files.iterrows():
        wave_path = os.path.join(path, row['file_name']+".pkl")
        st = op.read(wave_path)
        data_RFR = st.select(channel="RFR")[0].data
        data_RFT = st.select(channel="RFT")[0].data
        data_real[idx, :] = np.concatenate([data_RFR, data_RFT])
        baz.append(st[0].stats.baz)
        slow.append(st[0].stats.slow)
        idx += 1
    return data_real, baz, slow


def predict(baz, slow, thickn, rho, vp, vs, dip, strike, plunge, trend, ani, 
                   scale = 500, print_model = False, plot_model = False, plot_geom=True):
    _, _, result = all(baz, slow, thickn, rho, vp, vs, dip, strike, plunge, trend, ani, 426, 0.2,
                   scale = 500, print_model = False, plot_model = False, plot_geom=True)
    
    data_pred = np.zeros((len(result), 426*2))
    idx = 0
    for i in result:
        data = np.concatenate((i[1][0].data, i[1][1].data), axis = 0)
        data_pred[idx, :] = data
        idx += 1
    return data_pred
    

In [ ]:
layers = 3
data_real, baz, slow= read_all_data("CRLN")
n_waveforms = len(data_real)


thickn = [0 for thk in range(layers)]
rho = [0 for rho in range(layers)]
vp = [6000 for vp in range(layers)]
vs = [3500 for vs in range(layers)]
rho = [2800 for rho in range(layers)]
dip = [0 for dip in range(layers)]
strike = [0 for strike in range(layers)]
plunge = [0 for plunge in range(layers)]
trend = [0 for trend in range(layers)]
ani = [0 for ani in range(layers)]

thickn[0] = 70
thickn[1] = 3
rho[1] = 4000
rho[2] = 4000
vp[1] = 7000
vp[2] = 7500
vs[1] = 3800
vs[2] = 4000
dip[1] = 5
dip[2] = 5


In [ ]:

data_pred = predict(baz, slow, thickn, rho, vp, vs, dip, strike, plunge, trend, ani,
                   scale = 500, print_model = False, plot_model = False, plot_geom=True)

In [ ]:
plt.imshow(data_pred, aspect='auto')
plt.show()
plt.imshow(data_real, aspect='auto')
plt.show()